In [ ]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

In [5]:
master_reference_sheet_path = "master_sheet_reference.xlsx"
master_ref = pd.read_excel(master_reference_sheet_path, sheet_name=None, engine="openpyxl")

c:\Users\chavi.mangla\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\chavi.mangla\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\chavi.mangla\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\chavi.mangla\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\chavi.mangla\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be remov

In [ ]:
#checking if sheetnames are printed out
for sheet in master_ref:
    print(sheet)

P&M Schedule
P&M RIsk
Cost Estimation & Optimization
D&C - Quality
D&C - Procurement
D&C - HSE & Welfare
Innovation & Technology
Design & Technical
D&C - Construction
Strategy & Operations
Visuals Menu


In [12]:
ref_tabs = []

for sheet_name, df in master_ref.items():
    if {"Table", "Column"}.issubset(df.columns):
        table_cols = df.groupby("Table")["Column"].apply(list).to_dict()
        ref_tabs.append({sheet_name: table_cols})
    else:
        print(f"Skipping sheet {sheet_name}: Missing 'Table' or 'Column' column")

ref_tabs


Skipping sheet Visuals Menu: Missing 'Table' or 'Column' column


[{'P&M Schedule': {'AddDataPoint(P&M)': ['Additional Data Point',
    'Value',
    'Year',
    'Value Completed (Baseline) in M SAR',
    'Value Completed (Actual) in M SAR',
    'Value Completed (Forecast) in M SAR'],
   'DevCoAssessmentAnalysis(P&M)': ['Assessment Criteria', 'Value']}},
 {'P&M RIsk': {'AddDataPoint(P&M)': ['Additional Data Point',
    'Value',
    'Risk Category',
    'Number of Risk Items',
    'Risk',
    'Source',
    'Potential Impact',
    'Risk Rating',
    'Risk Status',
    'Current Measures'],
   'DevCoAssessmentAnalysis(P&M)': ['Assessment Criteria',
    'Value',
    'Rating']}},
 {'Cost Estimation & Optimization': {'AddDataPoint(CE&O)': ['Additional Data Point',
    'Value',
    'Year',
    'Budget Value (Baseline) in M SAR',
    'Budget Value (Revised) in M SAR',
    'Budget Value (Forecast) in M SAR'],
   'DevCoAssessmentAnalysis(CE&O)': ['Assessment Criteria', 'Value', 'Rating'],
   'DevCoAssessmentInput(CE&O)': ['Data Point',
    'Input Value',
    'Da

In [13]:
sheet_info = ref_tabs[0]
sheet_info

{'P&M Schedule': {'AddDataPoint(P&M)': ['Additional Data Point',
   'Value',
   'Year',
   'Value Completed (Baseline) in M SAR',
   'Value Completed (Actual) in M SAR',
   'Value Completed (Forecast) in M SAR'],
  'DevCoAssessmentAnalysis(P&M)': ['Assessment Criteria', 'Value']}}

In [17]:
sheet_name = list(sheet_info.keys())[0]  # Get the sheet name string
tables = sheet_info[sheet_name]          # Get dictionary of tables for this sheet
sheet_name
tables


{'AddDataPoint(P&M)': ['Additional Data Point',
  'Value',
  'Year',
  'Value Completed (Baseline) in M SAR',
  'Value Completed (Actual) in M SAR',
  'Value Completed (Forecast) in M SAR'],
 'DevCoAssessmentAnalysis(P&M)': ['Assessment Criteria', 'Value']}

In [ ]:
table_name = list(tables.keys())[0]  # Get first table name
columns = tables[table_name]          # List of columns for that table

columns

['Additional Data Point',
 'Value',
 'Year',
 'Value Completed (Baseline) in M SAR',
 'Value Completed (Actual) in M SAR',
 'Value Completed (Forecast) in M SAR']

In [ ]:
# for only one table in AG 
ag_df = pd.read_excel("250805 - Project LH - Assessment Guide Diriyah (SEC) V18.xlsx", sheet_name=table_name, engine="openpyxl", header=4)

In [ ]:
print(ag_df.columns.tolist()) # cols to a list conversion
print(ag_df.head(1))

['SN', 'Criteria', 'Additional Data Point', 'Value', 'Scorecard\nVisualization', 'Unnamed: 5', 'Year', 'Value Completed (Baseline) in M SAR', 'Value Completed (Actual) in M SAR', 'Value Completed (Forecast) in M SAR', 'Unnamed: 10', 'Risk', 'Source', 'Potential Impact', 'Risk Rating', 'Risk Status', 'Current Measures', 'Unnamed: 17', 'Risk Category', 'Number of Risk Items']
   SN  Criteria              Additional Data Point  Value  \
0   1       NaN  Latest Approved Completion Date​   Q2-28   

   Scorecard\nVisualization  Unnamed: 5    Year  \
0  Visualized with no score         NaN  2025.0   

   Value Completed (Baseline) in M SAR  Value Completed (Actual) in M SAR  \
0                                  0.0                                NaN   

   Value Completed (Forecast) in M SAR  Unnamed: 10    Risk      Source  \
0                                  NaN          NaN  Risk 1  Contractor   

  Potential Impact Risk Rating Risk Status Current Measures  Unnamed: 17  \
0             C

In [44]:
ag_df.columns = ag_df.columns.str.strip()
filtered_df = ag_df[[col for col in columns if col in ag_df.columns]]

In [45]:
output_path = f"Filtered_{table_name}.xlsx"
filtered_df.to_excel(output_path, index=False)
print(f"✅ Saved: {output_path}")

✅ Saved: Filtered_AddDataPoint(P&M).xlsx
